Declaration

In [220]:
import numpy as np
import os
import urllib.request
from sklearn import mixture
from scipy.spatial import distance
import math
import statistics 

CUT_OFF_IN_PERCENT = 0.02

# Clustering data set
# http://cs.joensuu.fi/sipu/datasets/
def download_data_set(file_url = 'http://cs.joensuu.fi/sipu/datasets/s1.txt'):
  local_filename, headers = urllib.request.urlretrieve(url = file_url)
  data = np.loadtxt(local_filename)
  dims = data.shape[1]
  if dims > 2:
    return np.delete(data, 2, 1)
  return data

def gmm_selection(data_set):
  lowest_bic = np.infty
  bic = []
  n_components_range = range(1, min(10, data_set.shape[0]))
  cv_types = ['spherical', 'tied', 'diag', 'full']
  for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(data_set)
        bic.append(gmm.bic(data_set))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
  # print(best_gmm)
  return best_gmm

# Get all euclid distance between points with keeping their respective order
def get_all_distances(data_set):
  arr_leng = data_set.shape[0]
  all_distances = []
  for i in range(arr_leng):
    current_point_distances = []
    for j in range(arr_leng):
      if i == j:
        continue
      current_point_distances.append(distance.euclidean(data_set[i], data_set[j]))
    all_distances.append(current_point_distances)
  return np.array(all_distances)

def get_cut_off_distance(distances_data):
  arr_leng, dims = distances_data.shape
  # Get number of neighbors
  number_of_items = math.ceil(CUT_OFF_IN_PERCENT * dims)
  all_distances = []
  for i in range(arr_leng):
    point_distance = np.array(distances_data[i])
    # Get number_of_items smallest distance
    point_distance = np.partition(point_distance,number_of_items)[:number_of_items]
    for dist in point_distance:
      all_distances.append(dist)
  return statistics.mean(all_distances) 

def get_density_of_point(point_distance, cut_off_distance):
  arr_leng = len(point_distance)
  result = 0
  for i in range(arr_leng):
    if point_distance[i] < cut_off_distance:
      result += 1
  return result

def get_densities(distances_data, cut_off_distance):
  arr_leng = distances_data.shape[0]
  result = []
  for i in range(arr_leng):
    result.append(get_density_of_point(distances_data[i], cut_off_distance))
  return np.array(result)

def get_distribution(data_set):
  unique, counts = np.unique(data_set, return_counts=True)
  arr_leng = unique.shape[0]
  result = []
  for i in range(arr_leng):
    result.append([unique[i], counts[i]])
  return np.array(result)

def get_density_groups(densities, densities_distribution, groups):
  result = []
  for i in range(len(densities)):
    current_density = densities[i]
    for j in range(len(densities_distribution)):
      if current_density == densities_distribution[j][0]:
        result.append(groups[j])
        break
  return np.array(result)
  
def get_neighbors(all_distances, current_point_index, cut_off_distance):
  neighbor_indexes = []
  neighbor_dinstances = []
  point_distances = all_distances[current_point_index]
  for i in range(len(point_distances)):
    if point_distances[i] < cut_off_distance:
      if i < current_point_index:
        neighbor_indexes.append(i)
      else:
        neighbor_indexes.append(i + 1)
      neighbor_dinstances.append(point_distances[i])
  return (neighbor_indexes, neighbor_dinstances)

def get_index_before_sort(sorted_index, argsort_arr):
  return argsort_arr[sorted_index]

def get_respective_index(index, argsort_arr):
  return argsort_arr[index]

def get_minimum_gaussian_groups(densities_distribution, groups):
  results = []
  for i in range(len(np.unique(groups))):
    results.append(0)
  for i in range(len(densities_distribution)):
    # print(str(groups[i]) + "" + str(densities_distribution[i]))
    group = groups[i]
    results[group] += densities_distribution[i][1]
  return np.where(results == min(results))[0]

def get_cluster_center_of_label(label, cluster_centers_label, cluster_centers):
  return cluster_centers[cluster_centers_label.index(label)]

Download data set

In [211]:
# Download data set
raw_data_set = download_data_set('http://cs.joensuu.fi/sipu/datasets/R15.txt')

Download data, calculate all relevant variables

In [212]:
# calculate all distances
all_distances = get_all_distances(raw_data_set)

# get cut off distance
cut_off_distance = get_cut_off_distance(all_distances)

# calculate densitive base on cut off distance
densities = get_densities(all_distances, cut_off_distance)

# form the distribution to perform gmm selection
densities_distribution = get_distribution(densities)

# get gmm models
gmm_model = gmm_selection(densities_distribution)

# get respective groups
groups = gmm_model.predict(densities_distribution)

# get a respective vector with original densities to represent the desitive of respective point
densities_groups = get_density_groups(densities, densities_distribution, groups)

# Referenced of the original array for futher classification point
densities_argsort = densities.argsort()[::-1]

# Copy to avoid sam referenced then sort in decending order
sorted_densities = np.array(densities)
sorted_densities.sort()
sorted_densities = sorted_densities[::-1]

# Calculate minimum gauss
minimum_gaussian_groups = get_minimum_gaussian_groups(densities_distribution, groups)

In [ ]:
# print(all_distances[0])
print(get_neighbors(all_distances, 0, cut_off_distance))
# for i in range(600):
  # print(len(get_neighbors(all_distances, i, cut_off_distance)))
print(raw_data_set[0])
print(densities_distribution)

In [ ]:
Algorithm

In [232]:
# sorted_densities cluster_centers = np.append(cluster_centers, 1)
data_set_length = len(raw_data_set)
cluster_centers = []
cluster_centers_label = []
labels = np.zeros((data_set_length,), dtype=np.int)
current_label = 0
print(densities_distribution)
print(groups)
print(densities_groups)
# for loop in range(data_set_length):
for loop in range(data_set_length):
  index_in_raw_data = get_respective_index(loop, densities_argsort)
  current_point_density = densities[index_in_raw_data]
  neighbor_indexes, neighbor_distances = get_neighbors(all_distances, index_in_raw_data, cut_off_distance)
  neighbors_has_clustered = []
  neighbors_has_clustered_distance = []
  neighbors_density_group = []
  neighbor_cluster_centers = []
  neighbor_cluster_centers_densities = []
  neighbor_cluster_centers_group = []
  for n_count in range(len(neighbor_indexes)):
    neighbor_index_in_sorted = get_respective_index(neighbor_indexes[n_count], densities_argsort)
    if labels[neighbor_index_in_sorted] != 0:
      neighbors_has_clustered.append(neighbor_index_in_sorted)
      neighbors_has_clustered_distance.append(neighbor_distances[n_count])
      neighbors_density_group.append(densities_groups[neighbor_indexes[n_count]])
      # Prepare data for further process by finding current neighbor's cluster center
      nbcc = get_cluster_center_of_label(labels[neighbor_index_in_sorted], cluster_centers_label, cluster_centers)
      neighbor_cluster_centers.append(nbcc)
      neighbor_cluster_centers_densities.append(densities[get_respective_index(nbcc, densities_argsort)])
      neighbor_cluster_centers_group.append(densities_groups[get_respective_index(nbcc, densities_argsort)])
  print("PHUUUUUU")
  print("Current density" + str(current_point_density))
  print("Current density group" + str(densities_groups[index_in_raw_data]))

  print(neighbor_cluster_centers)
  print(neighbor_cluster_centers_densities)
  print(neighbor_cluster_centers_group)
  print("============================")
  if len(neighbors_has_clustered) > 0:
    if len(neighbors_has_clustered) == 1:
      labels[loop] = labels[get_respective_index(neighbors_has_clustered[0], densities_argsort)]
    # All this point's neighbors belong to the same cluster
    elif len(neighbors_has_clustered) > 1 and len(set(neighbor_cluster_centers)) == 1:
      labels[loop] = labels[get_respective_index(neighbors_has_clustered[0], densities_argsort)]
    elif len(neighbors_has_clustered) > 1 and len(set(neighbor_cluster_centers)) > 1 and current_point_density in neighbor_cluster_centers_group:
      # Merging steps
      print("Merging")
    else:
      # If there are no comparative neighbor's cluster center with the same gaussian group with current point
      # Then get the cluster center of nearest neighbors and belong to that clusters
      decided_cluster = neighbors_has_clustered[neighbors_has_clustered_distance.index(min(neighbors_has_clustered_distance))]
      print(decided_cluster)
      labels[loop] = labels[decided_cluster]
  elif (not densities_groups[index_in_raw_data] in minimum_gaussian_groups) and sorted_densities[loop] > 0:
    # No neighbor then promote to cluster center
    cluster_centers.append(loop)
    current_label += 1
    # Keep respective cluster label with cluster centers
    cluster_centers_label.append(current_label)
    labels[loop] = current_label
  else:
    # Outlier point
    labels[loop] = 0
    continue

print(labels)
print(len(cluster_centers))

[[ 0 32]
 [ 1 35]
 [ 2 39]
 [ 3 48]
 [ 4 43]
 [ 5 36]
 [ 6 45]
 [ 7 36]
 [ 8 53]
 [ 9 41]
 [10 37]
 [11 39]
 [12 35]
 [13 32]
 [14 31]
 [15 11]
 [16  2]
 [17  1]
 [18  3]
 [19  1]]
[7 7 2 0 8 2 8 2 6 5 5 5 5 4 4 3 1 1 1 1]
[5 5 5 2 2 8 5 8 4 7 0 8 6 8 5 8 5 5 0 2 7 6 2 7 2 6 2 5 0 5 2 5 5 6 7 5 6
 8 5 7 5 2 6 5 5 4 2 2 2 5 7 2 5 6 5 5 8 2 8 2 8 0 5 5 8 2 7 7 2 5 2 6 3 8
 4 6 6 4 0 2 8 2 4 6 5 5 5 7 2 0 2 4 5 5 2 8 8 4 5 5 7 8 0 0 5 5 8 0 4 2 2
 7 2 0 5 2 8 5 0 2 8 2 8 0 2 6 7 7 6 2 2 7 7 6 2 2 6 5 2 6 8 2 8 7 2 6 0 6
 8 7 8 8 8 8 5 8 2 0 2 7 4 5 5 4 1 5 4 0 8 0 3 4 8 6 2 3 2 4 1 1 1 2 4 7 7
 2 4 5 7 5 8 0 4 4 5 5 8 6 4 8 0 2 4 4 8 2 7 7 7 2 4 2 5 5 6 8 3 6 5 5 0 0
 4 4 7 2 3 5 6 5 8 5 7 5 5 0 2 4 0 5 0 7 0 4 1 0 6 1 0 5 8 3 8 2 5 4 0 0 8
 4 8 5 2 8 2 2 4 5 6 6 2 8 5 7 5 4 8 3 2 4 6 5 7 5 2 8 7 2 4 2 6 5 0 7 8 7
 5 5 5 1 8 4 0 5 5 8 4 7 5 7 5 4 2 7 5 0 7 6 2 4 8 0 6 6 5 0 6 5 8 5 2 5 7
 2 8 5 6 7 6 5 5 7 5 2 4 5 5 7 2 6 5 4 5 6 5 4 6 8 5 5 0 2 4 5 2 2 5 5 3 4
 2 5 4 6 7 5 7 8 5 8 0 5 5 